# Model XGBoost - Attack Data

In [1]:
#Imports
import pandas as pd
import numpy as np

from eda_functions import split_data

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
# Read in cleaned attack file
attack_df = pd.read_csv('../data/attack_clean.csv')
attack_df.head()

,rev_id,comment,target
0,37675,This is not creative Those are the dictionar...,0
1,44816,the term standard model is itself less NPOV...,0
2,49851,True or false the situation as of March 2002...,0
3,89320,Next maybe you could work on being less conde...,0
4,93890,This page will need disambiguation,0


In [3]:
# Apply the custom train test split function to balance the classes in the training data only
X_train, X_test, y_train, y_test = split_data(
    attack_df,
    pct_positive=0.5,
    random_state=42)

In [4]:
# View the split for the train and test data
pd.DataFrame({
    f'Train (n={y_train.shape[0]})': y_train.value_counts(normalize=True),
    f'Test (n={y_test.shape[0]})': y_test.value_counts(normalize=True)})

,Train (n=21872),Test (n=34668)
0,0.5,0.866563
1,0.5,0.133437


In [5]:
# Set up a pipeline with CountVectorizer and XGBClassifier
pipe = Pipeline([
    ('cvec', CountVectorizer()),
    ('xgb', XGBClassifier())
])

In [6]:
# Set parameters
pipe_params = {
    'cvec__max_features': [4_000, 5_000, 6_000],
    'cvec__min_df'      : [2, 3],
    'cvec__max_df'      : [9, .95],
    'cvec__stop_words'  : ['english'],
    'cvec__token_pattern' : [r'\w+|[A-Z]\w+'],
    'cvec__strip_accents' : ['ascii'],
    'cvec__ngram_range' : [(1,1), (1,2)],
    'xgb__colsample_bytree': [0.5, 0.6, 0.7],
    'xgb__n_estimators': [100, 200, 250]
}

In [7]:
# Instatiate Gridsearch
gs = GridSearchCV(estimator = pipe,
                     param_grid = pipe_params,
                     cv = 5)

In [1]:
#gs.fit(X_train, y_train)
#uncomment this code to run gridsearch

In [9]:
gs.best_score_

0.8585863165458228

In [10]:
print(f'Train score: {gs.score(X_train, y_train)}')
print(f'Test score: {gs.score(X_test, y_test)}')

Train score: 0.9060442574981712
Test score: 0.9059363101419177


In [11]:
gs.best_params_

{'cvec__max_df': 0.95,
 'cvec__max_features': 6000,
 'cvec__min_df': 3,
 'cvec__ngram_range': (1, 1),
 'cvec__stop_words': 'english',
 'cvec__strip_accents': 'ascii',
 'cvec__token_pattern': '\\w+|[A-Z]\\w+',
 'xgb__colsample_bytree': 0.6,
 'xgb__n_estimators': 250}

In [12]:
# Set up a pipeline with TfidVectorizer and XGBClassifier
pipe_tfid = Pipeline([
    ('tvec', TfidfVectorizer()),
    ('xgb', XGBClassifier())
])

In [13]:
# Set parameters
pipe_tfid_params = {
    'tvec__max_features': [4_000, 5_000, 6_000],
    'tvec__min_df'      : [2, 3],
    'tvec__max_df'      : [9, .95],
    'tvec__stop_words'  : ['english'],
    'tvec__token_pattern' : [r'\w+|[A-Z]\w+'],
    'tvec__strip_accents' : ['ascii'],
    'tvec__ngram_range' : [(1,1), (1,2)],
    'xgb__colsample_bytree': [0.5, 0.75],
    'xgb__n_estimators': [100, 200, 300]
}

In [14]:
# Instatiate Gridsearch
gs_tfid = GridSearchCV(estimator = pipe_tfid,
                     param_grid = pipe_tfid_params,
                     cv = 5)

In [2]:
#gs_tfid.fit(X_train, y_train)
#uncomment this code to run gridsearch

In [16]:
gs_tfid.best_score_

0.8578547312038671

In [17]:
print(f'Train score: {gs_tfid.score(X_train, y_train)}')
print(f'Test score: {gs_tfid.score(X_test, y_test)}')

Train score: 0.9289502560351134
Test score: 0.9012922579900773


In [18]:
gs_tfid.best_params_

{'tvec__max_df': 0.95,
 'tvec__max_features': 5000,
 'tvec__min_df': 3,
 'tvec__ngram_range': (1, 1),
 'tvec__stop_words': 'english',
 'tvec__strip_accents': 'ascii',
 'tvec__token_pattern': '\\w+|[A-Z]\\w+',
 'xgb__colsample_bytree': 0.75,
 'xgb__n_estimators': 300}